<a href="https://colab.research.google.com/github/GUILLENJV/Optimizaci-n-de-Campa-as/blob/master/D_TG_RandomSerch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
#Destroy Old Oracle
!rm -rf tuning-mlp

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D,Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.random import set_seed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import pickle
import kerastuner as kt
import keras_tuner as kt


<ipython-input-3-a98c80d582dc>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
from numpy import unique
import numpy
import numpy as np
from numpy import array
import pandas as pd
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
from tensorflow.keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot
import seaborn as sns
from keras.utils import to_categorical

In [ ]:
df = pd.read_csv("62f1ab79cc287b001f643389_clear_final.csv")
X = df[['campaign_id',
        'traffic_source_id',
        'visitor_device_browser',
        'visitor_device_hardware_family',
        'visitor_device_hardware_model',
        'visitor_device_hardware_vendor',
        'visitor_device_os_family',
        'visitor_device_os_vendor',
        'visitor_device_os_version',
        'visitor_device_type',
        'visitor_geo_location_cityName',
        'visitor_geo_location_connection_typ',
        'visitor_geo_location_countryCode',
        'visitor_geo_location_isp',
        'visitor_geo_location_regionName',
        'visitor_tokens_adh',
        'visitor_tokens_cadid',
        'visitor_tokens_adi',
        'converted_yes',
        'converted_no']]
y = df[["landing_page_id"]]

df = pd.concat([X, y], axis=1)

# prepare input data
def prepare_inputs(X_train, X_test):
	oe = OneHotEncoder()
	oe.fit(X_train)
	X_train_enc = oe.transform(X_train)
	X_test_enc = oe.transform(X_test)
	return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
  le = LabelEncoder()
  le.fit(y_train)
  y_train_enc = le.transform(y_train)
  y_test_enc = le.transform(y_test)
  y_train_enc = to_categorical(y_train_enc)
  y_test_enc = to_categorical(y_test_enc)
  return y_train_enc, y_test_enc

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)

# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
### Tunable Model
def mlp_model_builder(hp):
  model = Sequential()

  model.add(Dense(units = hp.Int('dense-bot', min_value=100, max_value=250,
                                 step=16), input_shape=(246,)))

  for i in range(hp.Int('num_dense_layers', 1, 2, 3)):
    model.add(Dense(units=hp.Int('dense_' + str(i),
                                 min_value=100, max_value=250, step=16),
                     activation='relu'))
    model.add(Dropout(hp.Choice('dropout_'+ str(i),
                                values=[0.0, 0.1, 0.2, 0.4])))
    model.add(Dense(2,activation="sigmoid"))

    hp_optimizer=hp.Choice('Optimizer', values=['Adam', 'SGD', 'RMSprop',
                                              'Adadelta', 'Adagrad', 'Adamax',
                                              'Nadam'])
    hp_activation=hp.Choice('Activation', values=['softplus', 'softsign', 'relu', 'tanh', 'sigmoid'])

  if hp_optimizer == 'Adam':
      hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
  elif hp_optimizer == 'SGD':
      hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
      nesterov=True
      momentum=0.9

  model.compile( optimizer=hp_optimizer,loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
# set random seed
random_seed = 8
tf.random.set_seed = random_seed
np.random.seed(random_seed)

In [ ]:
# Overload arguments with our custom metric.
tuner_mlp = kt.tuners.RandomSearch(
    mlp_model_builder,
    seed=random_seed,
    objective='val_loss',
    max_trials=30,
    directory='.',
    project_name='tuning-mlp')

In [ ]:
### Search Space
tuner_mlp.search(X_train_enc, y_train_enc, epochs=10, batch_size=64, validation_data=(X_test_enc, y_test_enc), shuffle=False)

Trial 30 Complete [00h 00m 16s]
val_loss: 0.059687383472919464

Best val_loss So Far: 0.058343954384326935
Total elapsed time: 00h 09m 42s


In [ ]:
best_mlp_hyperparameters = tuner_mlp.get_best_hyperparameters(1)[0]
print("Best Hyper-parameters")
best_mlp_hyperparameters.values

Best Hyper-parameters


{'dense-bot': 180,
 'num_dense_layers': 1,
 'dense_0': 244,
 'dropout_0': 0.1,
 'Optimizer': 'Nadam',
 'Activation': 'softplus',
 'learning_rate': 0.01}

In [ ]:
#Define Callbacks
early_stop = EarlyStopping(monitor='val_loss', restore_best_weights=True,
                                   patience=5, verbose=1)

callback = [early_stop]

In [ ]:
model_mlp = Sequential()
model_mlp.add(Dense(best_mlp_hyperparameters['dense-bot'], input_shape=(246,),
                    activation='relu'))
for i in range(best_mlp_hyperparameters['num_dense_layers']):
  model_mlp.add(Dense(units=best_mlp_hyperparameters['dense_' +str(i)],
                      activation='relu'))
  model_mlp.add(Dropout(rate=best_mlp_hyperparameters['dropout_' +str(i)]))
  model_mlp.add(Dense(2,activation="sigmoid"))

  model_mlp.compile(optimizer=best_mlp_hyperparameters['Optimizer'],
                                                 loss='binary_crossentropy',
                                                 metrics=['accuracy'])
history_mlp= model_mlp.fit(X_train_enc, y_train_enc, epochs=50, batch_size=32,
                           validation_data=(X_test_enc, y_test_enc))


ValueError: Unknown activation function 'Activation' cannot be deserialized.

In [ ]:
print(model_mlp.summary())

In [ ]:
# Plotting functions

def plot_history(hs, epochs, metric):
    plt.rcParams['font.size'] = 16
    plt.figure(figsize=(15, 8))
    for label in hs:
        plt.plot(hs[label].history[metric], label='{0:s} train {1:s}'.format(label, metric), linewidth=2)
        plt.plot(hs[label].history['val_{0:s}'.format(metric)], label='{0:s} validation {1:s}'.format(label, metric), linewidth=2)
    #x_ticks = np.arange(0, epochs + 1, epochs / 10)
    #x_ticks [0] += 1
    #plt.xticks(x_ticks)
    plt.ylim((0, 1))
    plt.xlabel('Epochs')
    plt.ylabel('Loss' if metric=='loss' else 'Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(
    hs={
        'Best MLP': history_mlp
    },
    epochs=25,
    metric='loss')

plot_history(
    hs={
        'Best MLP': history_mlp
    },
    epochs=25,
    metric='accuracy')

In [ ]:
#Calculate accuracies
mlp_train_loss, mlp_train_acc = model_mlp.evaluate(X_train_enc,  y_train_enc, verbose=2)
print('\nTrain accuracy:', mlp_train_acc)

#mlp_dev_loss, mlp_dev_acc = model_mlp.evaluate(dev_x,  dev_y, verbose=2)
#print('\nValidation accuracy:', mlp_dev_acc)

mlp_test_loss, mlp_test_acc = model_mlp.evaluate(X_test_enc,  y_test_enc, verbose=2)
print('\nTest accuracy:', mlp_test_acc)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.clf()
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(15,8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    plt.xlim(-0.5, len(np.unique(classes))-0.5)
    plt.ylim(len(np.unique(classes))-0.5, -0.5)
    return ax

In [ ]:
class_names = ['0', '1']

In [ ]:
predictions=model_mlp.predict(X_test_enc)

plot_confusion_matrix(tf.argmax(predictions, axis = 1).numpy(), tf.argmax(y_test_enc, axis = 1).numpy(),  classes=class_names,
                      title='Confusion matrix on best MLP results')
plt.show()

In [ ]:
predictions